Data: https://zenodo.org/records/10946767

The goal of the project is to create a deep learning network to predict DNA accessibility across multiple Arabidopsis experiments. The data is available on zenodo. It contains both raw read coverage files (aka BigWig files) and peak files in BED format, like the files you used in Assignment 3.  For the project, consider the problem as a regression problem, i.e. your task is to predict predict read coverage rather than the presence of peaks.

The zenodo repository also includes a metadata spreadsheet that indicates the source of the biological samples (the project ID and Accession number columns) as well as the plant tissue that was used to generate the samples.

In training and evaluating your models, we suggest you use chromosomes 1-4 for training and validation and chromosome 5 for testing.

In your project we expect you to evaluate different architectures (e.g. purely convolutional vs transformer), explore them in terms of depth and other aspects of the design (e.g. regulation and other features such as layer normalization), and perform an analysis of the filters learned by the network.  The objective is for you to develop some intuition of what works or doesn't work in this domain.

In designing your approach we recommend carefully studying the approach used in the Basenji paper that will be discussed in class.  The following paper is another useful resource:

Toneyan, S., Tang, Z. & Koo, P.K. Evaluating deep learning for predicting epigenomic profiles. Nature Machine Intelligence 4, 1088–1100 (2022).  https://doi.org/10.1038/s42256-022-00570-9


Goal
Predict DNA accessibility sites across different Arabidopsis experiments.

Problem Framing:
Given input DNA sequence, predict read coverage as a continuous, quantitative response variable.

Data
Raw Data
Raw read coverage local filepaths (similar to those from Bassenji)
Local filepaths of (BED) peaks (like those from Assignment 3)
Arabidopsis genome

Metadata
Source of biological samples (project ID and Accession)
Plant tissue identifier

Training Data
Chromosomes 1-4 will be randomly split 80/20 into train and validation data. Chromosome 5 will be held out as a test set.

Data Loading
Load in the multiple Arabidopsis experiments read coverage data, and Arabidopsis genome
Generate testing, training, and validation set generators
Use generator objects that will get the one hot encoded training, testing, and validation datasets so that the datasets can be randomized for each run

Output: List of sequences, and coverage map of those sequences

Biological Datasets
Biologically relevant parts of the genome will be curated into datasets to explore how the models perform with known biological functions using annotations. All of these annotations will hopefully be available on ENCODE or other online resources.

- Promoter dataset
- Enhancer dataset
- CTCF dataset 


Architecture
We are proposing 3 different model architectures based on what we have discussed in class:

- Basset model
Small input sequence length
3 convolutional filters

- Bassenji model
Large input sequence length (10s of kb)
4 convolutional filters + 5 dilated convolutional filters (Arabidopsis has a ~10x smaller genome than humans)

- Bassenji model with transformers
Use positional encoding + multi-head attention layer

Hyperparameters
There are various hyperparameters that we aim to experiment with. Since we are predicting a continuous variable with a regression function, we will use a Poisson regression loss function, as done in the Bassenji model. We may look into GPyOpt (https://github.com/SheffieldML/GPyOpt) for hyperparameter optimization, but will likely just experiment with the hyperparameters manually.
Hyper Parameters to Test (not all hyper parameters apply to all networks):
Learning rate
Number of layers
Batch size
Convolutional filter size
Number of convolutional filters
Input dropout rate (to inform performance on noisy data)
Dropout rate
Num. attention heads
Input layer size
Read length

Prediction
Our prediction is that the Bassenji model will be the best performing, followed by the basic Bassenji, followed by the Basset. Since we have already implemented something similar to the Basset model, this will serve as a useful benchmark.

Biological Interpretation
We aim to provide a rigorous interpretation of the biological significance of our model results, taking into account performance across different cell types, optimal read length, optimal convolutional filter size and number of filters, and other relevant aspects of model architecture.


Goal:
Predict DNA accessibility sites across different Arabidopsis experiments.

Problem Framing:
Given input DNA sequence, predict read coverage as a continuous, quantitative response variable.

Data:

Raw Data

Raw read coverage local filepaths (similar to those from Bassenji)
Local filepaths of (BED) peaks (like those from Assignment 3)
Arabidopsis genome

Metadata

Source of biological samples (project ID and Accession)
Plant tissue identifier

Training Data

Chromosomes 1-4 will be randomly split 80/20 into train and validation data. Chromosome 5 will be held out as a test set.

Data Loading

Load in the multiple Arabidopsis experiments read coverage data, and Arabidopsis genome
Generate testing, training, and validation set generators
Use generator objects that will get the one hot encoded training, testing, and validation datasets so that the datasets can be randomized for each run

Output: List of sequences, and coverage map of those sequences

Biological Datasets

Biologically relevant parts of the genome will be curated into datasets to explore how the models perform with known biological functions using annotations. All of these annotations will hopefully be available on ENCODE or other online resources.

Promoter dataset

Enhancer dataset

CTCF dataset 


Human introns: 1500bp
Arabidopsis: 150bp

Basset: 600bp
Bassenji: 130kb
Window length: 2.5kb

Human genome: 3B bases
Arabidopsis: 100mb

36 outputs (for each bigwig file)

Predict single value for each 2.5kb segment

Since peaks can be very high

Each label is a 36 dimensional vector

In [1]:
import collections
import glob
import gzip
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyBigWig
import random
import scipy.signal

from Bio import SeqIO

from sklearn import metrics
from sklearn.model_selection import train_test_split

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(42);

import torchvision
from torchvision.transforms import ToTensor

device = (
    "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
#device = "cpu"

import load_data

/Users/tedmonyak/miniconda3/envs/gp/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/tedmonyak/miniconda3/envs/gp/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/tedmonyak/miniconda3/envs/gp/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/tedmonyak/miniconda3/envs/gp/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/tedmonyak/miniconda3/envs/gp/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/tedmonyak/miniconda3/envs/gp/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/tedmonyak/miniconda3/envs/gp/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be

Using mps device


In [2]:
# Create a fasta file from the bigwig file
# Generates sequences of length bin_size, sliding a window of size interval across
# each chromosome
# Assumes that chr_fnames already exists
def generate_input_files_from_bw(bw_fname,
                                 output_fasta,
                                 output_faste,
                                 seq_length=2500,
                                 interval=1250):
    bw = pyBigWig.open(bw_fname)
    chrs = ['Chr1', 'Chr2', 'Chr3', 'Chr4', 'Chr5']
    output_fasta = open(output_fasta, "w")
    output_faste = open(output_faste, "w")
    for chr_id in chrs:
        chr_fname = chr_fnames[chr_id]
        with gzip.open(chr_fname, "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                chr_seq = str(record.seq)
                chr_len = bw.chroms(chr_id)
                bw_idx = 0
                while bw_idx + seq_length < chr_len:
                    coverage = ",".join(map(str, bw.values(chr_id, bw_idx, bw_idx + seq_length)))
                    seq = chr_seq[bw_idx:bw_idx + seq_length]
                    seq_id =  ",".join([chr_id, str(bw_idx), str(bw_idx+seq_length)])
                    output_fasta.write(">" + seq_id + "\n" + seq + "\n")
                    output_faste.write(">" + seq_id + "\n" + coverage + "\n")
                    bw_idx += interval

In [ ]:
base_dir = '../project/chromatin_cs425'
chr_fnames = {'Chr1': '../project/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz',
              'Chr2': '../project/Arabidopsis_thaliana.TAIR10.dna.chromosome.2.fa.gz',
              'Chr3': '../project/Arabidopsis_thaliana.TAIR10.dna.chromosome.3.fa.gz',
              'Chr4': '../project/Arabidopsis_thaliana.TAIR10.dna.chromosome.4.fa.gz',
              'Chr5': '../project/Arabidopsis_thaliana.TAIR10.dna.chromosome.5.fa.gz'}

input_dirs = [os.path.join(base_dir, 'SRP034156'), os.path.join(base_dir, 'SRP300093')]

bw_fname = os.path.join(base_dir, 'SRP034156', 'SRX1096548_Rep0.rpgc.bw')

generate_input_files_from_bw(bw_fname,
                             '../project/chromatin_cs425/SRP034156/fasta/SRP034156.fasta',
                             '../project/chromatin_cs425/SRP034156/fasta/SRP034156.faste')

In [3]:
def get_data_loaders(data_dir, train_size, test_size, batch_size=64):
    train_dataset, val_dataset, test_dataset = load_data.load_data(data_dir,
                                                                   train_val_data_to_load=train_size,
                                                                   test_data_to_load=test_size)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                              batch_size=batch_size,shuffle=True)
    test_loader = DataLoader(dataset=test_dataset,
                              batch_size=batch_size,shuffle=True)
    return train_loader, val_loader, test_loader

In [ ]:
train_loader, val_loader, test_loader = get_data_loaders('../project/chromatin_cs425/SRP034156/fasta',
                                                         train_size=math.inf,
                                                         test_size=math.inf)

In [ ]:
class DnaCnn(nn.Module):
    def __init__(self, num_kernels=[20, 32, 32], kernel_size=[12,12,12],
                 dropout=0):
        super(DnaCnn, self).__init__()
        self.input_channels=4
        self.num_kernels=num_kernels
        self.kernel_size=kernel_size
        self.dropout=dropout
        self.conv_block = nn.Sequential(
            # first layer
            nn.Conv1d(in_channels=self.input_channels,
                      out_channels=num_kernels[0],
                      kernel_size=kernel_size[0]),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.MaxPool1d(kernel_size=2),
        )
        # second layer
        self.conv_block.append(nn.Sequential(
            nn.Conv1d(in_channels=self.num_kernels[0],
                      out_channels=num_kernels[1],
                      kernel_size=kernel_size[1]),
            nn.ReLU(),
            #nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=self.dropout),            
        ))
        # Add a third convolutional layer
        self.conv_block.append(nn.Sequential(
            # second layer
            nn.Conv1d(in_channels=self.num_kernels[1],
                      out_channels=num_kernels[2],
                      kernel_size=kernel_size[2]),
            nn.ReLU(),
            #nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=self.dropout),            
        ))
        self.regression_block = nn.Sequential(
            nn.Linear(num_kernels[2], num_kernels[2]),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),            
            nn.Linear(num_kernels[2], 1)
            #nn.Sigmoid()
        )            

    def forward(self, x):
        x = self.conv_block(x)
        x,_ = torch.max(x, dim=2)        
        x = self.regression_block(x)
        return x

In [ ]:
def train_epoch(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss = 0
    # set the model to training mode - important when you have 
    # batch normalization and dropout layers
    model.train()
    for batch_idx, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        # Compute prediction and loss
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        # backpropagation
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 10 == 0 :
        print(f"training loss: {total_loss/num_batches:>7f}")
    return total_loss / num_batches

def validation(dataloader, model, loss_fn, epoch):
    # set the model to evaluation mode 
    model.eval()
    # size of dataset
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    validation_loss, correct = 0, 0
    # Evaluating the model with torch.no_grad() ensures that no gradients 
    # are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage 
    # for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            validation_loss += loss_fn(y_pred, y).item()
    validation_loss /= num_batches
    if epoch%10 == 0 :
        print(f"Validation Loss: {validation_loss:>8f} \n")
    return validation_loss

In [ ]:
def train_model():
    epochs = 50
    loss_fn = nn.PoissonNLLLoss()
    patience = 10
    
    train_loss = []
    validation_loss = []
    best_loss = math.inf
    for t in range(epochs):
        if t % 10 == 0 :
            print(f"Epoch {t}\n-------------------------------")
        loss = train_epoch(train_loader, model, loss_fn, optimizer, t)
        train_loss.append(loss)
        loss = validation(val_loader, model, loss_fn, t)
        validation_loss.append(loss)
    
        if loss < best_loss:
            best_loss = loss    
            p = patience
        else:
            p -= 1
            if p == 0:
                print("Early Stopping!")
                break    
    print("Done!")

    def plot_loss():
        plt.figure(figsize=(4,3))
        plt.plot(train_loss, label='Training')
        plt.plot(validation_loss, label='Validation')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.ylim(0)
        plt.legend();

    plot_loss()

In [ ]:
model = DnaCnn().to(device)
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_model()